<a href="https://colab.research.google.com/github/Akashnawin/Mainframe-ML-Capstone/blob/main/Mainframe_Firebase_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mainframe-firebase-integration-firebase-adminsdk-fbsvc-d08d5a8675.json to mainframe-firebase-integration-firebase-adminsdk-fbsvc-d08d5a8675.json


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate("mainframe-firebase-integration-firebase-adminsdk-fbsvc-d08d5a8675.json")
firebase_admin.initialize_app(cred)

db = firestore.client()
print("Firebase connected successfully!")


Firebase connected successfully!


In [ ]:
uploaded = files.upload()

Saving tbl_msg_synthetic.csv to tbl_msg_synthetic.csv


In [ ]:
import pandas as pd

df = pd.read_csv("tbl_msg_synthetic.csv")
df.head()

,SEQNO,SYSID,JOBNAME,MSGID,TIMESTMP,USERID,TEXT,LABEL
0,1,MVSF,USER325,IEF403I,09.50.16,OPSS,-WTO2,normal
1,2,MVSF,OPSS,EZZ6034E,09.02.02,OPSS,ERROR: Job STEP1 ABEND RC=16 - URGENT,critical
2,3,MVSF,PM0,IEF196I,09.44.21,SYSOP,EZZ6034I TN3270 CONN 000585F0 LU MULTIPLE CONN...,normal
3,4,MVSF,PM0,IEF403I,09.28.51,USER325M,OPS TRAINING: NORMAL MSG 1 - SYSTEM OK,normal
4,5,MVSF,USER325M,IEF403I,09.06.36,USER325M,OPS TRAINING: NORMAL MSG 2 - JUST CHECKING IN,normal


In [ ]:
collection_name = "tbl_msg"

for index, row in df.iterrows():
    doc_id = str(row["SEQNO"])   # each row becomes its own document
    db.collection(collection_name).document(doc_id).set(row.to_dict())

print("Upload complete!")

Upload complete!


In [ ]:
!pip install -q firebase-admin pandas scikit-learn matplotlib seaborn wordcloud joblib

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import os

# If firebase already initialized in this notebook you can skip re-init.
# Otherwise:
# cred = credentials.Certificate("firebase-key.json")
# firebase_admin.initialize_app(cred)

db = firestore.client()
COLL = 'tbl_msg'

def fetch_all(collection):
    docs = db.collection(collection).stream()
    rows = []
    for d in docs:
        rec = d.to_dict()
        rec['__docid'] = d.id
        rows.append(rec)
    return pd.DataFrame(rows)

df = fetch_all(COLL)
print("Fetched rows:", len(df))
df.head(5)


Fetched rows: 800


,MSGID,TIMESTMP,USERID,LABEL,JOBNAME,TEXT,SEQNO,SYSID,__docid
0,IEF403I,09.50.16,OPSS,normal,USER325,-WTO2,1,MVSF,1
1,EZZ6034I,09.25.54,USER325M,normal,USER325M,-WTO2,10,MVSF,10
2,PMO396I,09.03.43,SYSOP,normal,OPSS,-WTO1,100,MVSF,100
3,PREFIX,09.00.50,SYSOP,normal,JES2,-WTOR2,101,MVSF,101
4,IEF403I,09.58.49,START1,normal,CONSOLE,-WTO1,102,MVSF,102
